In [ ]:
Daily Reports Automation

In [37]:
import boto3
#import pandas as pd
import os
import zipfile

In [2]:
#start a session
session = boto3.Session(profile_name='default',region_name='us-west-2')

In [6]:
s3_client = session.client('s3')

In [35]:
#Check if the bucket exists

def bucket_exists(bucket_name):
    # s3_client = boto3.client('s3')

    response = s3_client.list_buckets()
    buckets = [bucket['Name'] for bucket in response.get('Buckets', [])]
    # print (f'List of buckets: {buckets}')
    return bucket_name in buckets

bucket_name = 'daily-reports-05-06-2025'
bucket_exists (bucket_name)

if bucket_exists(bucket_name):
    print(f'Bucket {bucket_name} exists.')
else:
    s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={
        'LocationConstraint': session.region_name
    }    
    )
    print(f'Bucket {bucket_name} created in {session.region_name} region')

Bucket daily-reports-05-06-2025 exists.


In [ ]:
# check if folder exists
folder_name = 'daily-reports-05-06-2025/'
folders_in_bucket = []
files_in_bucket = []
objects_in_bucket =  []

response = s3_client.list_objects_v2( Bucket=bucket_name , Prefix=folder_name)

if 'Contents' in response:
    print(f"Folder '{folder_name}' already exists in bucket '{bucket_name}'")
# Create 'folder' (empty object with the folder name as key)
else:
    s3_client.put_object(
                Bucket=bucket_name,
                Key=folder_name,
                Body=''
            )
    print(f"Created folder '{folder_name}' in bucket '{bucket_name}'")
           

Folder 'daily-reports-05-06-2025/' already exists in bucket 'daily-reports-05-06-2025'
Folders in bucket daily-reports-05-06-2025: []
Files in bucket daily-reports-05-06-2025: []
Files in bucket daily-reports-05-06-2025: []
{'ResponseMetadata': {'RequestId': 'RXDWTF8SFRPTYKP8', 'HostId': '/B/oE07idMahehTLx5MB8PDvf9D4LUP5t/tT+fShm02fe9GLJzNJMX+tmFKQVgCnluO6nBx6+lU=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '/B/oE07idMahehTLx5MB8PDvf9D4LUP5t/tT+fShm02fe9GLJzNJMX+tmFKQVgCnluO6nBx6+lU=', 'x-amz-request-id': 'RXDWTF8SFRPTYKP8', 'date': 'Tue, 06 May 2025 07:42:47 GMT', 'x-amz-bucket-region': 'us-west-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'daily-reports-05-06-2025/', 'LastModified': datetime.datetime(2025, 5, 6, 6, 48, 47, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'Sto

In [ ]:
def create_zip_file(output_filename):
    # getting the list of files to zip
    files_to_zip = []
    for _, _, files in os.walk(local_path):
        for file in files:
            files_to_zip.append(local_path+file)
    # Create a zip file and add files to it
    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in files_to_zip:
            if os.path.isfile(file):
                zipf.write(file, os.path.basename(file))
            else:
                print(f"Warning: {file} not found or not a file")
    print (f'Files in local path {local_path}: {files_to_zip}')


local_path = './test_folder/'
zip_file = './dayly_zip/archive.zip'

create_zip_file(zip_file)
# print (files_to_zip)


Files in local path ./test_folder/: ['./test_folder/text1.txt', './test_folder/text2.txt', './test_folder/text3.txt']


In [52]:
def upload_zip_to_s3(zip_file, bucket_name, folder_name):
    # Upload the zip file to S3
    s3_client.upload_file(zip_file, bucket_name, folder_name + os.path.basename(zip_file))
    print(f"Uploaded {zip_file} to s3://{bucket_name}/{folder_name}{os.path.basename(zip_file)}")
upload_zip_to_s3(zip_file, bucket_name, folder_name)

Uploaded ./dayly_zip/archive.zip to s3://daily-reports-05-06-2025/daily-reports-05-06-2025/archive.zip


In [ ]:
# Upload the zip file to S3

if 'Contents' in response:
        for bucket in response['Contents']:
            objects_in_bucket.append(bucket['Key'])

            

def upload_files_to_bucket(bucket_name, local_directory):
    # files in bucket
    response = s3_client.list_objects_v2( Bucket=bucket_name )
    files_in_bucket = []
    if 'Contents' in response:
        for bucket in response['Contents']:
            files_in_bucket.append(bucket['Key'])
    
    # files in local directory
    file_in_local = []
    for _, _, files in os.walk('./daily_documents'):
        for file in files:
            file_in_local.append(file)
    # check if the files in local directory are already in the bucket
    print ('Starting upload of daily documents')
    counter = 0
    for file in file_in_local:
        if file in files_in_bucket:
            # file_in_local.remove(file)
            print (f'File {file} already exists in the bucket')
            
        else:
            s3_client.upload_file(local_file_name, bucket_name, s3_file_name)
            print (f'File {local_file_name} uploaded to bucket {bucket_name} with name {s3_file_name}')
            counter += 1
            

    print ('Upload of daily documents completed')
    print (f'Total number of files uploaded: {counter}')


local_directory= 'daily_documents'
upload_files_to_bucket(bucket_name, local_directory)